Importing all libraries and data needed

In [1]:
import numpy as np
import pandas as pd
import folium
import time
import openrouteservice as ors
from linearProgram import optimal_Routes
from linearProgramSat import optimal_routes_weekend

C:\Users\jh220\anaconda\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
Status: Optimal


Reading in coordinates of stores

In [2]:
locations = pd.read_csv("WoolworthsLocations.csv")
cords = locations[['Long' , 'Lat']]
locs = locations['Store']
coords = cords.to_numpy().tolist()
cords_locs = locations[['Store','Long','Lat']]

Setting up optimal routes

In [3]:
OptR = optimal_Routes
OptRW = optimal_routes_weekend

Route Visualisations for weekdays

In [50]:
#My ORSKey
ORSKey= '5b3ce3597851110001cf6248f1e943d6f27f45f38a3d16b210d02c68'
client = ors.Client(key=ORSKey)
colors = ['#00FFFF','#458B74','#C1CDCD','#E3CF57','#000000','#0000FF','#8A2BE2','#FF4040','#8A360F','#FF6103',
         '#7FFF00','#458B00','#8B4513','#3D59AB','#FF7256','#FFF8DC','#DC143C','#008B8B','#9932CC','#C1FFC1',
         '#EE1289','#808080','#FF69B4','#8470FF','#00FA9A','#8B5A00','#33A1C9','#FF0000','#F4A460','#8E8E38']

m = folium.Map(location = [-36.949,174.808],zoom_start = 10)
timer = 0
#Loop through every optimal route
for i in range(len(OptR)):
    #array to store the coordinates of the stores
    #Initial node/store which is the distribution centre
    a = [(174.808,-36.949)]
    timer = timer + 1
    if (timer == 20):
        time.sleep(60)
        timer = 0
    #For every node in the route get the coords and append it to the array
    #Turn the list into a tuple because the client wants it in tuple form. 
    for r in OptR[i]:
        x = cords_locs.index[cords_locs['Store'] == r].tolist()
        c = tuple((coords[x[0]]))
        a.append(c)
        route = client.directions(
            coordinates = a,
            profile = "driving-hgv",
            format = "geojson",
            validate = False
        )
        
        folium.PolyLine(locations = [list(reversed(coord))
                                    for coord in 
                                        route['features'][0]['geometry']['coordinates']],color = colors[i], opacity = 0.8).add_to(m)
        
for i in range(0, len(coords)):
    if locations.Type[i] == "Countdown":
        iconCol = "green"
    elif locations.Type[i] == "FreshChoice":
        iconCol = "blue"
    elif locations.Type[i] == "SuperValue":
        iconCol = "red"
    elif locations.Type[i] == "Countdown Metro":
        iconCol = "orange"
    elif locations.Type[i] == "Distribution Centre":
        iconCol = "black"
    folium.Marker(list(reversed(coords[i])), popup= locations.Store[i], icon = folium.Icon(color = iconCol)).add_to(m)
m

Route Visualisations for Saturday

In [53]:
m = folium.Map(location = [-36.949,174.808],zoom_start = 10)
#Implement a timer so the rate limit for getting directions is not exceeded
timer = 0
#Loop through every optimal route
for i in range(len(OptRW)):
    #array to store the coordinates of the stores
    #Initial node/store which is the distribution centre
    a = [(174.808,-36.949)]
    timer = timer + 1
    if (timer == 9):
        time.sleep(60)
        timer = 0
    #For every node in the route get the coords and append it to the array
    #Turn the list into a tuple because the client wants it in tuple form. 
    for r in OptRW[i]:
        x = cords_locs.index[cords_locs['Store'] == r].tolist()
        c = tuple((coords[x[0]]))
        a.append(c)
        route = client.directions(
            coordinates = a,
            profile = "driving-hgv",
            format = "geojson",
            validate = False
        )
        
        folium.PolyLine(locations = [list(reversed(coord))
                                    for coord in 
                                        route['features'][0]['geometry']['coordinates']],color = colors[i], opacity = 0.8).add_to(m)
        
for i in range(0, len(coords)):
    if locations.Type[i] == "Countdown":
        iconCol = "green"
    elif locations.Type[i] == "FreshChoice":
        iconCol = "blue"
    elif locations.Type[i] == "SuperValue":
        iconCol = "red"
    elif locations.Type[i] == "Countdown Metro":
        iconCol = "orange"
    elif locations.Type[i] == "Distribution Centre":
        iconCol = "black"
    folium.Marker(list(reversed(coords[i])), popup= locations.Store[i], icon = folium.Icon(color = iconCol)).add_to(m)
m